In [2]:
import geopandas as gpd
import folium
import numpy as np
from scipy.spatial import distance_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from sklearn.cluster import DBSCAN

# Load GeoJSON data
gdf = gpd.read_file('updated_export.geojson')

# Extract tower nodes
tower_nodes = [(row['id'], (row['geometry'].y, row['geometry'].x)) for index, row in gdf.iterrows() if 'power' in row and row['power'] == 'tower']

# Create a list of node positions for distance calculations
positions = [coords for _, coords in tower_nodes]

# Calculate the distance matrix
dist_matrix = distance_matrix(positions, positions)

def generate_random_connections_with_in_out_constraint(num_nodes):
    nodes = list(range(num_nodes))
    np.random.shuffle(nodes)

    out_edges = set()
    in_edges = set()

    # Connect each node to another ensuring each node has one outgoing link
    for i in range(0, num_nodes - 1, 2):
        out_edges.add((nodes[i], nodes[i + 1]))
    if num_nodes % 2 != 0:
        out_edges.add((nodes[-1], nodes[np.random.choice(num_nodes - 1)]))
    
    # Connect each node to another ensuring each node has one incoming link and avoid duplicate links
    shuffled_nodes = nodes.copy()
    np.random.shuffle(shuffled_nodes)
    for i in range(0, num_nodes - 1, 2):
        if (shuffled_nodes[i], shuffled_nodes[i + 1]) not in out_edges and (shuffled_nodes[i + 1], shuffled_nodes[i]) not in out_edges:
            in_edges.add((shuffled_nodes[i], shuffled_nodes[i + 1]))
        else:
            in_edges.add((shuffled_nodes[i + 1], shuffled_nodes[i]))
    if num_nodes % 2 != 0:
        for j in nodes:
            if (shuffled_nodes[-1], j) not in out_edges and (j, shuffled_nodes[-1]) not in out_edges and shuffled_nodes[-1] != j:
                in_edges.add((shuffled_nodes[-1], j))
                break

    # Combine in and out edges
    all_edges = list(out_edges.union(in_edges))
    return all_edges

# Run the modified function for 20 generations
best_configs = []

NUM_GENERATIONS = 10000
CONFIGS_PER_GENERATION = 300

for generation in range(NUM_GENERATIONS):
    generation_configs = []
    for _ in range(CONFIGS_PER_GENERATION):
        random_edges = generate_random_connections_with_in_out_constraint(len(positions))
        total_distance = sum(dist_matrix[i][j] for i, j in random_edges)
        generation_configs.append((total_distance, random_edges))
    
    # Get the most fit configuration in this generation
    best_config_in_generation = min(generation_configs, key=lambda x: x[0])
    best_configs.append(best_config_in_generation)

# Get the most fit configuration from all generations
most_fit_config = min(best_configs, key=lambda x: x[0])

# Create a new map to display the most fit configuration
# Create a folium map centered at the mean coordinates
mean_coords = gdf.geometry.unary_union.centroid
m_best = folium.Map(location=[mean_coords.y, mean_coords.x], zoom_start=10)

# Add original tower nodes to the map
for _, coords in tower_nodes:
    folium.Marker(location=coords, icon=folium.Icon(icon='bolt')).add_to(m_best)

# Add the most fit configuration's connections to the map
for i, j in most_fit_config[1]:
    locs = [tower_nodes[i][1], tower_nodes[j][1]]
    folium.PolyLine(locs, color="blue", weight=2.5, opacity=1, tooltip="Best Configuration").add_to(m_best)

# Create a folium map centered at the mean coordinates
m_best